In [3]:
import sys
import os
# Add the src directory to Python's search path
sys.path.append(os.path.join(os.getcwd(), 'src'))

In [4]:
%pwd

'c:\\Users\\donga\\OneDrive\\Documents\\ML_Projects\\HousePricePredictor\\research'

In [5]:
os.chdir("c:\\Users\\donga\\OneDrive\\Documents\\ML_Projects\\HousePricePredictor")

In [6]:
%pwd

'c:\\Users\\donga\\OneDrive\\Documents\\ML_Projects\\HousePricePredictor'

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_URL : str
    local_data_file: Path
    unzip_dir: Path

In [8]:
from src.housePricePrediction.constants import *
from src.housePricePrediction.utils.common import read_yaml, create_directories

In [14]:
from pathlib import Path

CONFIG_FILE_PATH = Path("config/config.yaml")
PARAMS_FILE_PATH = Path("params.yaml")
SCHEMA_FILE_PATH = Path("schema.yaml")

class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

        # Set up artifacts_root
        # self.artifacts_root = self.config.get("artifacts_root", "./artifacts")
    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [21]:
import os 
import urllib.request as request
import tarfile
import zipfile
from housePricePrediction import logger
from housePricePrediction.utils.common import get_size

In [22]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

        
    def extract_zip_file(self):
        """ 
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        
        os.makedirs(self.config.unzip_dir, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(self.config.unzip_dir)

    def extract_tgz_file(self):
        """ 
        Extracts a .tgz file to specific directory.
        """
        with tarfile.open(self.config.local_data_file,'r:gz') as tar:
            tar.extractall(path= self.config.unzip_dir)
        

In [26]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_tgz_file()
except Exception as e:
    raise e

[2024-11-23 11:22:37,864 : INFO: common : yaml file: config\config.yaml loaded successfully]
[2024-11-23 11:22:37,867 : INFO: common : yaml file: params.yaml loaded successfully]
[2024-11-23 11:22:37,871 : INFO: common : yaml file: schema.yaml loaded successfully]
[2024-11-23 11:22:37,872 : INFO: common : created directory at: artifacts]
[2024-11-23 11:22:37,874 : INFO: common : created directory at: artifacts/data_ingestion]
[2024-11-23 11:22:37,875 : INFO: 3354594285 : File already exists of size: ~439KB]
